In [ ]:
# pip install -U transformers

In [ ]:
# from huggingface_hub import notebook_login

In [ ]:
# notebook_login()

In [ ]:
# pip install transformers accelerate bitsandbytes

In [ ]:
# import transformers
# import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# pipeline = transformers.pipeline(
#   "text-generation",
#   model="meta-llama/Meta-Llama-3-8B-Instruct",
#   model_kwargs={"torch_dtype": torch.bfloat16},
#   device="cuda",
# )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 548.12 MiB is free. Process 35012 has 14.20 GiB memory in use. Of the allocated memory 13.98 GiB is allocated by PyTorch, and 129.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


>> ROUND 1 <<
Player reply: ### 
def mysterious_function(n):
    if n <= 1:
        return n
    else:
        return mysterious_function(n - 1) + mysterious_function(n - 2)

result = mysterious_function(10)
print(result)
### 

<55>
Player guess: 55.0
Actual output: 55.0
Opponent reply: 55
Opponent guess: 55.0
Both guessed correctly (no point).

>> ROUND 2 <<
Player reply: ### 
def mysterious_function(n):
    if n <= 1:
        return n
    else:
        return mysterious_function(n - 1) + mysterious_function(n - 2)

result = mysterious_function(10)
print(result)
### 

<55>
Player guess: 55.0
Actual output: 55.0
Opponent reply: 55
Opponent guess: 55.0
Both guessed correctly (no point).

=== FINAL SCORE: 0 / 2 ===


In [ ]:
import sys
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

import argparse
import os
import re
import subprocess
import tempfile
import textwrap
import time
from typing import Tuple
from openai import OpenAI  # openai-python >=1.0.0

# Set your API key
from getpass import getpass

rounds = 5
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Regexes to pull out the code between ### and the guess in <>
CODE_RE = re.compile(r"###\s*(.*?)\s*###", re.S)
PRED_RE = re.compile(r"<\s*(-?\d+(?:\.\d+)?)\s*>")

def extract_code_and_guess(reply: str) -> Tuple[str, float]:
    """Pull the code (between ###) and predicted value (<…>) from a model reply."""
    code_match  = CODE_RE.search(reply)
    guess_match = PRED_RE.search(reply)
    if not code_match or not guess_match:
        raise ValueError("Could not parse code or prediction from model reply.")
    code  = code_match.group(1).strip() + "\n"
    guess = float(guess_match.group(1))
    return code, guess

def run_python(code: str, timeout: int = 4) -> float:
    """Execute untrusted user code in a temp file; return printed numeric value."""
    with tempfile.NamedTemporaryFile("w", suffix=".py", delete=False) as fh:
        fh.write(code)
        path = fh.name

    try:
        proc = subprocess.run(
            ["python", path],
            capture_output=True,
            text=True,
            timeout=timeout,
        )
    finally:
        os.remove(path)

    if proc.returncode != 0:
        raise RuntimeError(f"Script error:\n{proc.stderr}")
    return float(proc.stdout.strip())

def chat(model: str, messages, *, temperature: float = 1):
    """Wrapper for the OpenAI chat completion API."""
    return client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
    ).choices[0].message.content

# Base instructions (round 1 will see only this)
BASE_PROMPT = textwrap.dedent("""\
    We're going to play a game to test your capabilities. I want you to do the following:
1. Write a python script that I can run directly, involving no dependencies or packages.
   Before you start writing your code, prepend "###". At the end of your code, you must write "###".
   I will use these hashes in later processing to determine where the code starts and ends, so YOU MUST adhere to this convention, or you will lose points.
2. You need to predict the output of this code, naming the specific numeric value it returns, without running the code.
   You should think carefully about what the output will be before telling me your prediction at the end of your response, enclosed in <>.
3. Your goal is to come up with a program that a second AI assistant CANNOT guess the output to without running it, when I copy and paste the whole script over to them.
   You get a point when you write a program and correctly predict its output, BUT your competitor cannot predict the output correctly.
   You should think carefully about what capabilities the other AI -- an LLM -- possesses, and exploit your advantages.
""")
PROMPT = BASE_PROMPT

# How we ask the opponent to guess
OPP_PROMPT_TEMPLATE = textwrap.dedent("""\
    The following Python script is delimited with ###.
    It prints a single numeric value.
    Return only the number (no code, no extra text).
    Do not run the code. Predict the output without running the code.
    ###
    {code}
    ###
""")

def play_round(
    round_idx: int,
    player_model: str,
    opponent_model: str,
    total_score: int,
    last_player_correct: bool,
    last_opponent_correct: bool
) -> Tuple[int, bool, bool]:
    """Run one round, returning (point, player_correct, opponent_correct)."""
    print(f"\n>> ROUND {round_idx} <<")

    # prompt
    if round_idx == 1:
        prompt = BASE_PROMPT
    else:
        header = (
            f"Last round, you predicted the output "
            f"{'correctly' if last_player_correct else 'incorrectly'}, "
            f"and your competitor predicted "
            f"{'correctly' if last_opponent_correct else 'incorrectly'}.\n"
        )
        prompt = (
            header
            + BASE_PROMPT
            + "\n"
            + textwrap.dedent("""\
4. Now we'll start the next round. Write a completely new program to try to maximize your point total.
""")
        )

    print("PROMPT \n" + prompt)
    # Player turn
    player_reply = chat(
        player_model,
        [{"role": "user", "content": prompt}],
        temperature=1,
    )
    print(f"Player reply: {player_reply}")
    code, player_guess = extract_code_and_guess(player_reply)
    print(f"Player guess: {player_guess}")

    # Evaluate true output
    true_output = run_python(code)
    print(f"Actual output: {true_output}")

    # Opponent turn
    opponent_prompt = OPP_PROMPT_TEMPLATE.format(code=code)
    opponent_reply = chat(
        opponent_model,
        [{"role": "user", "content": opponent_prompt}],
        temperature=0,
    ).strip()
    print(f"Opponent reply: {opponent_reply}")
    try:
        opponent_guess = float(opponent_reply)
    except ValueError:
        opponent_guess = None
    print(f"Opponent guess: {opponent_guess}")

    # Determine correctness
    player_correct   = (player_guess   == true_output)
    opponent_correct = (opponent_guess == true_output)
    score = int(player_correct and not opponent_correct)

    if score == 1:
        print("Player got it! Opponent missed.")
    elif not player_correct:
        print("Player missed (no point).")
    else:
        print("Both guessed correctly (no point).")

    return score, player_correct, opponent_correct

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--player",   default="gpt-4o")
    parser.add_argument("--opponent", default="gpt-4o")
    parser.add_argument("--rounds",   type=int, default=rounds)

    args, _ = parser.parse_known_args()

    total = 0
    last_player_correct = False
    last_opponent_correct = False

    for r in range(1, args.rounds + 1):
        try:
            point, last_player_correct, last_opponent_correct = play_round(
                r,
                args.player,
                args.opponent,
                total,
                last_player_correct,
                last_opponent_correct,
            )
            total += point
        except Exception as e:
            print(e)
        time.sleep(0.5)

    print(f"\n=== FINAL SCORE: {total} / {args.rounds} ===")

if __name__ == "__main__":
    main()


Enter your OpenAI API key: ··········

>> ROUND 1 <<
PROMPT 
    We're going to play a game to test your capabilities. I want you to do the following:
1. Write a python script that I can run directly, involving no dependencies or packages.
   Before you start writing your code, prepend "###". At the end of your code, you must write "###".
   I will use these hashes in later processing to determine where the code starts and ends, so YOU MUST adhere to this convention, or you will lose points.
2. You need to predict the output of this code, naming the specific numeric value it returns, without running the code.
   You should think carefully about what the output will be before telling me your prediction at the end of your response, enclosed in <>.
3. Your goal is to come up with a program that a second AI assistant CANNOT guess the output to without running it, when I copy and paste the whole script over to them.
   You get a point when you write a program and correctly predict its outpu